In [4]:
import xarray as xr
import glob
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.stats import pearsonr

In [5]:
#Select study domain
lat_min, lat_max = 55.38, 56.01
lon_min, lon_max = 8.15, 9.97

In [6]:
validation_dataset = xr.open_dataset('O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Era5-Land\\Hourly_Data\\Precipitation\\era5_land_1950_2024_tp_daily_update_DK_regridCORDEX.nc')
validation_dataset = validation_dataset.rename({"tp": "pr"})
validation_dataset['pr'] = validation_dataset['pr'] *1000

select_subset= (
    (validation_dataset['lat'] >= lat_min) & (validation_dataset['lat'] <= lat_max) &
    (validation_dataset['lon'] >= lon_min) & (validation_dataset['lon'] <= lon_max)
)

validation_dataset = validation_dataset.where(select_subset).dropna(dim="y", how="all").dropna(dim="x", how="all")

In [9]:
unique_model_names = glob.glob('O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\*.nc')

In [10]:
unique_model_names

['O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_1951-2099_BC_Esbjerg.nc',
 'O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_MOHC-HadREM3-GA7-05_1952-2098_BC_Esbjerg.nc',
 'O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-CCLM4-8-17_1951-2099_BC_Esbjerg.nc',
 'O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_SMHI-RCA4_1970-2099_BC_Esbjerg.nc',
 'O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_KNMI-RACMO22E_1951-2099_BC_Esbjerg.nc',
 'O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_ICTP-RegCM4-6_1981

In [11]:
len(unique_model_names)

6

In [12]:
# Open multiple files and concatenate along 'time' dimension
validation_ds = validation_dataset

# Select data 
validation_ds_selected = validation_ds.sel(
time=(
    (validation_ds["time"].dt.year >= 2011) &  
    (validation_ds["time"].dt.year <= 2024)  
),
drop=True  # Remove any missing data in the time range
)

val_annual_precipitation_sum = validation_ds_selected.resample(time="YE").mean()
val_annual_precipitation_mean = val_annual_precipitation_sum.mean(dim="time")
val_annual_precipitation_spatial_mean = val_annual_precipitation_mean.mean(dim=["y", "x"])

val_monthly_precipitation_sum = validation_ds_selected.resample(time="ME").mean()
val_monthly_precipitation_spatial_mean = val_monthly_precipitation_sum.groupby("time.month").mean(dim="time").mean(dim=["y", "x"])


In [13]:
model_outputs = {}

In [14]:
%%time
for model in list(unique_model_names):
    print(model)
    # Open multiple files and concatenate along 'time' dimension
    pr_ds = xr.open_dataset(model)

    # Select data 
    pr_ds = pr_ds.sel(
    time=(
        (pr_ds["time"].dt.year >= 2011) &  
        (pr_ds["time"].dt.year <= 2024)  
    ),
    drop=True  # Remove any missing data in the time range
    )

    #Indicator I (Relative difference in mean annual pecipitation)
    model_annual_precipitation_sum = pr_ds.resample(time="YE").mean()
    model_annual_precipitation_mean = model_annual_precipitation_sum.mean(dim="time")
    model_annual_precipitation_spatial_mean = model_annual_precipitation_mean.mean(dim=["y", "x"])
    model_annual_rel_difference = abs(((model_annual_precipitation_spatial_mean - val_annual_precipitation_spatial_mean)/val_annual_precipitation_spatial_mean).pr.values.item()*100)
    
    #Indicator II (Mean absolute monthly deviation)
    model_monthly_precipitation_sum = pr_ds.resample(time="ME").mean()
    model_monthly_precipitation_spatial_mean = model_monthly_precipitation_sum.groupby("time.month").mean(dim="time").mean(dim=["y", "x"])
    model_monthly_difference = 100 - np.abs((((np.abs(model_monthly_precipitation_spatial_mean - val_monthly_precipitation_spatial_mean).sum()/12)-val_annual_precipitation_spatial_mean)/val_annual_precipitation_spatial_mean).pr.values.item()*100)
    
    #Indicator III (Monthly temporal pattern)
    #Calculate standard deviation ratio of monthly values
    s_model = model_monthly_precipitation_spatial_mean.std()
    s_cal = val_monthly_precipitation_spatial_mean.std()
    s = s_cal / s_model
    s = s.pr.values
    
    #Calculate correlation of monthly values
    r, _= pearsonr(model_monthly_precipitation_spatial_mean.pr.values.flatten(), 
                   val_monthly_precipitation_spatial_mean.pr.values.flatten())
    
    #Calculate KGE (Indicator III)
    KGE = 1-((r-1)**2 + (s-1)**2)
    
    mask = ~np.isnan(model_annual_precipitation_mean.pr.values.flatten())
    
    #Indicator IV (Spatial correlation of annual means)
    correlation, _ = pearsonr(model_annual_precipitation_mean.pr.values.flatten()[mask],
                              val_annual_precipitation_mean.pr.values.flatten()[mask])
    
    # Store the spatial mean in the dictionary
    model_outputs[model] = {
        'Indicator I': model_annual_rel_difference,
        'Indicator II': model_monthly_difference,
        'Indicator III' : KGE,
        'Indicator IV' : correlation
    }
    
    print(f"Saved {model} outputs.")
    


O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_1951-2099_BC_Esbjerg.nc
Saved O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_1951-2099_BC_Esbjerg.nc outputs.
O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_MOHC-HadREM3-GA7-05_1952-2098_BC_Esbjerg.nc
Saved O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_MOHC-HadREM3-GA7-05_1952-2098_BC_Esbjerg.nc outputs.
O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-CCLM4-8-17_1951-2099_BC_Esbjerg.nc
Saved O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-CCLM4-8-17_1951-2099_BC_Esbjerg.nc outp

In [15]:
model_outputs

{'O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_1951-2099_BC_Esbjerg.nc': {'Indicator I': 0.6953710754285948,
  'Indicator II': 12.889039983343338,
  'Indicator III': 0.7905371713802337,
  'Indicator IV': 0.9756712719190817},
 'O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_MOHC-HadREM3-GA7-05_1952-2098_BC_Esbjerg.nc': {'Indicator I': 2.8722274451684577,
  'Indicator II': 13.262605307139111,
  'Indicator III': 0.8883291956386292,
  'Indicator IV': 0.9815288038801369},
 'O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\new_bc\\datetime\\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-CCLM4-8-17_1951-2099_BC_Esbjerg.nc': {'Indicator I': 9.872992480717032,
  'Indicator II': 17.412125146665815,
  'Indicator III': 0.7694743098532957,
  'Indicator IV': 0.9857293890723025},
 'O:\\Man\\Public\\sharing

In [16]:
#Save model_outputs to pandas dataframe
model_outputs_df = pd.DataFrame.from_dict(model_outputs, orient='index')

In [17]:
model_outputs_df

,Indicator I,Indicator II,Indicator III,Indicator IV
O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_GERICS-REMO2015_1951-2099_BC_Esbjerg.nc,0.695371,12.889040,0.790537,0.975671
O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_MOHC-HadREM3-GA7-05_1952-2098_BC_Esbjerg.nc,2.872227,13.262605,0.888329,0.981529
O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_CLMcom-CCLM4-8-17_1951-2099_BC_Esbjerg.nc,9.872992,17.412125,0.769474,0.985729
O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_SMHI-RCA4_1970-2099_BC_Esbjerg.nc,3.835890,10.083637,0.937345,0.989361
O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_KNMI-RACMO22E_1951-2099_BC_Esbjerg.nc,1.226116,9.996677,0.944005,0.989338
O:\Man\Public\sharing-4270-CERM\MATNEW\Klimaatlas_CORDEX\raw data\new_bc\datetime\EUR-11_MOHC-HadGEM2-ES_rcp85_r1i1p1_ICTP-RegCM4-6_1981-2099_BC_Esbjerg.nc,2.765221,14.293107,0.866918,0.973652


In [15]:
# Save to Excel
model_outputs_df.to_csv("O:\\Man\\Public\\sharing-4270-CERM\\MATNEW\\Klimaatlas_CORDEX\\raw data\\CORDEX_pr_indicators.csv")